In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# ! mkdir -p plant-pathology-2021-fgvc8
# ! unzip -q ../input/cut500-350 -d plant-pathology-2021-fgvc8

In [ ]:
import os
import numpy as np
import pandas as pd
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
# train = pd.read_csv('/content/drive/MyDrive/01python工研院課程/專題/data/train.csv')

train.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer().fit(train.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mb.transform(train.labels.apply(lambda x : x.split())), columns = mb.classes_)
new_train = pd.concat([train['image'], labels], axis=1)
new_train.head()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
pre = ImageDataGenerator(
    rescale = 1/255.,
    rotation_range = 15,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    zoom_range = 0.2,
    shear_range=0.3,
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split= 0.2,)
pos = ImageDataGenerator(rescale = 1./255)

In [ ]:

submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
import cv2
img_counts = submission.shape[0]

for i in range(img_counts):
    img = cv2.imread('../input/plant-pathology-2021-fgvc8/test_images/' + submission.loc[i,'image'])
    img = cv2.resize(img,(500,350))
    #cv2.imshow('img'+str(i),img)
    b,g,r = cv2.split(img)

    kernel_size = 5
    blur = cv2.GaussianBlur(g,(kernel_size, kernel_size), 0)
    blur = cv2.GaussianBlur(blur,(kernel_size, kernel_size), 0)
    
    edges = cv2.Canny(blur, 30, 200)
    #cv2.imshow('edges'+str(i),edges)
    edge_box = []
    for x in range(edges.shape[0]):
        for y in range(edges.shape[1]):
            if edges[x][y] != 0:
                edge_box.append((x, y))
                
    row_min = edge_box[np.argsort([box[0] for box in edge_box])[0]][0]
    row_max = edge_box[np.argsort([box[0] for box in edge_box])[-1]][0]
    col_min = edge_box[np.argsort([box[1] for box in edge_box])[0]][1]
    col_max = edge_box[np.argsort([box[1] for box in edge_box])[-1]][1]

    test_img= img[row_min:row_max, col_min:col_max]
    #cv2.imshow('cut'+str(i),new_img)
    test_img_resize = cv2.resize(test_img,(512,512))
    
    path = './test_img_500_350'
    if not os.path.isdir(path):
        os.makedirs(path)
    
    cv2.imwrite( './test_img_500_350/' + submission.loc[i,'image'],test_img_resize)

In [ ]:
seed = 88
target_size = (500, 350)
batch_size = 16
resize_img = '../input/cut500-350/cut_500x350'
test_img = './test_img_500_350'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
train_generator = pre.flow_from_dataframe(
                  new_train,
                  directory = resize_img, 
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "training",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)
valid_generator = pre.flow_from_dataframe(
                  new_train,
                  directory = resize_img,
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "validation",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)
test_generator = pos.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = None,
                  batch_size=8,
                  target_size = target_size,
                  color_mode="rgb",
                  shuffle = False,
                  seed = seed,)

In [ ]:
import matplotlib.pyplot as plt
example = next(train_generator)
print(example[0].shape)
plt.imshow(example[0][0,:,:,:])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
example = next(test_generator)
print(example[0].shape)
plt.imshow(example[0])
plt.show()

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.applications import EfficientNetB4
# from tensorflow.keras import layers, optimizers
# from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
# from keras import Model
# base_model = EfficientNetB4(include_top=False, 
#                 weights='imagenet',
#                 pooling='avg', 
#                 input_shape=(500,350,3))

In [ ]:
# model_pre = tf.keras.Sequential([
#         base_model,
#         layers.BatchNormalization(),
#         layers.Dense(32,activation='relu'),
#         layers.Dropout(0.4),
#         layers.Dense(16,activation='relu'),
#         layers.Dropout(0.4),
#         layers.Dense(6,activation='sigmoid')
# ])

In [ ]:
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# model_pre.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, decay=0.001/20),
#     loss='binary_crossentropy',
#     metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),tf.keras.metrics.AUC(name='auc')])
# model_checkpoint = ModelCheckpoint(
#             filepath="/content/drive/MyDrive/01python工研院課程/專題/data/0714_500_350_B4.h5", 
#             mode='max',
#             monitor='val_auc', 
#             save_best_only=True, 
#             save_freq="epoch", 
#             verbose=1)
# early_stopping = EarlyStopping(
#             monitor='val_loss',
#             mode='min', 
#             min_delta=0.003,
#             patience=10, 
#             verbose=1, 
#             restore_best_weights=True)
# model_pre = model_pre.fit(train_generator,
#             validation_data=valid_generator,
#             # steps_per_epoch=1000,
#             epochs=30,
#             callbacks=[model_checkpoint, early_stopping])

# WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. 
# Available metrics are: loss,binary_accuracy,auc,val_loss,val_binary_accuracy,val_auc

In [ ]:
# import matplotlib.pyplot as plt

# pre_history = model_pre.history

# plt.figure()
# plt.plot(pre_history['binary_accuracy'])
# plt.plot(pre_history['val_binary_accuracy'])
# plt.title('model binary_accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'])
# plt.savefig('accuracy')
# plt.show()

In [ ]:
# pre_history = model_pre.history

# plt.figure()
# plt.plot(pre_history['auc'])
# plt.plot(pre_history['val_auc'])
# plt.title('model auc')
# plt.ylabel('auc')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'])
# plt.savefig('auc')
# plt.show()

In [ ]:
# plt.figure()
# plt.plot(pre_history['loss'])
# plt.plot(pre_history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'])
# plt.savefig('loss')
# plt.show()

In [ ]:
import keras
def load_model():
    model_pre = keras.models.load_model("../input/0714-500-350-b4/0714_500_350_B4.h5")
    return model_pre
model = load_model()

In [ ]:
# base_model.trainable = True
# ###
# from keras.callbacks import ModelCheckpoint, EarlyStopping

# model_checkpoint = ModelCheckpoint(
#             filepath="/content/drive/MyDrive/01python工研院課程/專題/data/0714_500_350_B4.h5", 
#             mode='max',
#             monitor='val_auc', 
#             save_best_only=True, 
#             save_freq="epoch", 
#             verbose=1)
# early_stopping = EarlyStopping(
#             monitor='val_loss',
#             mode='min', 
#             min_delta=0.003,
#             patience=10, 
#             verbose=1, 
#             restore_best_weights=True)
# ###
# model.compile(
#     optimizer=keras.optimizers.Adam(1e-5, decay=1e-5/10),
#     loss=keras.losses.BinaryCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),tf.keras.metrics.AUC(name='auc')])
# model = model.fit(
#     train_generator,
#     validation_data=valid_generator,
#     epochs=10,
#     callbacks=[model_checkpoint, early_stopping])

In [ ]:
# model_history = model.history

# plt.figure()
# plt.plot(model_history['binary_accuracy'])
# plt.plot(model_history['val_binary_accuracy'])
# plt.title('model binary_accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'])
# plt.savefig('accuracy')
# plt.show()

In [ ]:
model = load_model()
pred = model.predict(test_generator)

In [ ]:
perdict = (pred>0.40)
n_label = new_train.columns.tolist()[1:]
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
# np.around(pred, decimals=１, out=None)
pred

In [ ]:
submission['labels'] = np.array(answer)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
# !mv /content/content/drive/MyDrive/FGVC8/python工研院課程/專題/data/0713test_B4.h5 /content/drive/MyDrive/FGVC8/python工研院課程/專題/data